In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import calendar
from datetime import date
%matplotlib inline
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from plotly import graph_objs as go

In [6]:
df=pd.read_csv("data/train.csv",parse_dates=["Order Date","Ship Date"])

/var/folders/0b/3m1h5_mj5q57nkn5qzvf49cr0000gn/T/ipykernel_2275/3879490915.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df=pd.read_csv("data/train.csv",parse_dates=["Order Date","Ship Date"])
/var/folders/0b/3m1h5_mj5q57nkn5qzvf49cr0000gn/T/ipykernel_2275/3879490915.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df=pd.read_csv("data/train.csv",parse_dates=["Order Date","Ship Date"])


In [7]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,2017-08-11,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,2017-08-11,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,2017-12-06,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,2016-11-10,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-11-10,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9800 non-null   int64         
 1   Order ID       9800 non-null   object        
 2   Order Date     9800 non-null   datetime64[ns]
 3   Ship Date      9800 non-null   datetime64[ns]
 4   Ship Mode      9800 non-null   object        
 5   Customer ID    9800 non-null   object        
 6   Customer Name  9800 non-null   object        
 7   Segment        9800 non-null   object        
 8   Country        9800 non-null   object        
 9   City           9800 non-null   object        
 10  State          9800 non-null   object        
 11  Postal Code    9789 non-null   float64       
 12  Region         9800 non-null   object        
 13  Product ID     9800 non-null   object        
 14  Category       9800 non-null   object        
 15  Sub-Category   9800 n

In [9]:
df = df.drop(['Row ID','Customer ID','Order ID','Customer Name'],axis=1)

In [10]:
df.head()

,Order Date,Ship Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,2017-08-11,2017-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2017-08-11,2017-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2017-12-06,2017-06-16,Second Class,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,2016-11-10,2016-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,2016-11-10,2016-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [11]:
df.duplicated().sum()

1

In [12]:
#print duplicate rows
print(df[df.duplicated()])

     Order Date  Ship Date       Ship Mode      Segment        Country  \
3406 2015-04-23 2015-04-27  Standard Class  Home Office  United States   

          City State  Postal Code Region       Product ID   Category  \
3406  Columbus  Ohio      43229.0   East  FUR-CH-10002965  Furniture   

     Sub-Category                                       Product Name    Sales  
3406       Chairs  Global Leather Highback Executive Chair with P...  281.372  


In [13]:
df = df.drop_duplicates()

In [14]:
df.head()

,Order Date,Ship Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,2017-08-11,2017-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2017-08-11,2017-11-11,Second Class,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2017-12-06,2017-06-16,Second Class,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,2016-11-10,2016-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,2016-11-10,2016-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [15]:
df.isnull().sum()

Order Date       0
Ship Date        0
Ship Mode        0
Segment          0
Country          0
City             0
State            0
Postal Code     11
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
dtype: int64

In [16]:
df['Postal Code'].fillna(5401,inplace=True)

In [17]:
df.isnull().sum().sum()

0

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9799 entries, 0 to 9799
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order Date    9799 non-null   datetime64[ns]
 1   Ship Date     9799 non-null   datetime64[ns]
 2   Ship Mode     9799 non-null   object        
 3   Segment       9799 non-null   object        
 4   Country       9799 non-null   object        
 5   City          9799 non-null   object        
 6   State         9799 non-null   object        
 7   Postal Code   9799 non-null   float64       
 8   Region        9799 non-null   object        
 9   Product ID    9799 non-null   object        
 10  Category      9799 non-null   object        
 11  Sub-Category  9799 non-null   object        
 12  Product Name  9799 non-null   object        
 13  Sales         9799 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(10)
memory usage: 1.1+ MB


In [19]:
#print different countries
print(df['Country'].unique())

['United States']


In [20]:
#drop country column
df.drop('Country',axis=1,inplace=True)

In [21]:
#drop postal code column and ship date column
df.drop(['Postal Code','Ship Date'],axis=1,inplace=True)

In [22]:
df.drop(['Ship Mode','Product ID'],axis=1,inplace=True)

In [23]:
#print the latest order date

df['Order Date'] = pd.to_datetime(df['Order Date'].astype(str), format='%Y-%m-%d')




In [24]:
print(df['Order Date'].max())

print(df['Order Date'].min())

2018-12-30 00:00:00
2015-01-02 00:00:00


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9799 entries, 0 to 9799
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order Date    9799 non-null   datetime64[ns]
 1   Segment       9799 non-null   object        
 2   City          9799 non-null   object        
 3   State         9799 non-null   object        
 4   Region        9799 non-null   object        
 5   Category      9799 non-null   object        
 6   Sub-Category  9799 non-null   object        
 7   Product Name  9799 non-null   object        
 8   Sales         9799 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 765.5+ KB


In [26]:
df.head()

,Order Date,Segment,City,State,Region,Category,Sub-Category,Product Name,Sales
0,2017-08-11,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2017-08-11,Consumer,Henderson,Kentucky,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2017-12-06,Corporate,Los Angeles,California,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,2016-11-10,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,2016-11-10,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [27]:
df.to_csv('data/cleaned_data.csv',index=False)

In [28]:
print(df['Order Date'].max())

2018-12-30 00:00:00


In [29]:
df_cleaned = pd.read_csv("data/cleaned_data.csv")

In [30]:
df_cleaned

,Order Date,Segment,City,State,Region,Category,Sub-Category,Product Name,Sales
0,2017-08-11,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2017-08-11,Consumer,Henderson,Kentucky,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2017-12-06,Corporate,Los Angeles,California,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,2016-11-10,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,2016-11-10,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...
9794,2017-05-21,Corporate,Chicago,Illinois,Central,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9795,2016-12-01,Corporate,Toledo,Ohio,East,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9796,2016-12-01,Corporate,Toledo,Ohio,East,Technology,Phones,GE 30524EE4,235.1880
9797,2016-12-01,Corporate,Toledo,Ohio,East,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [31]:
#divide the data into two parts from 2014 to 2017 and 2018

df_train = df[df['Order Date'] < '2018-01-01']

df_test = df[df['Order Date'] >= '2018-01-01']

In [32]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6541 entries, 0 to 9799
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order Date    6541 non-null   datetime64[ns]
 1   Segment       6541 non-null   object        
 2   City          6541 non-null   object        
 3   State         6541 non-null   object        
 4   Region        6541 non-null   object        
 5   Category      6541 non-null   object        
 6   Sub-Category  6541 non-null   object        
 7   Product Name  6541 non-null   object        
 8   Sales         6541 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 511.0+ KB


In [33]:
# plot the sales trend for the training data set per year using plotly

df_train['year'] = df_train['Order Date'].dt.year

df_train['month'] = df_train['Order Date'].dt.month

# plot only for years
df_yearly = df_train.groupby('year')['Sales'].sum().reset_index()



 

/var/folders/0b/3m1h5_mj5q57nkn5qzvf49cr0000gn/T/ipykernel_2275/2123951092.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['year'] = df_train['Order Date'].dt.year
/var/folders/0b/3m1h5_mj5q57nkn5qzvf49cr0000gn/T/ipykernel_2275/2123951092.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['month'] = df_train['Order Date'].dt.month


In [34]:
df_yearly.head()

,year,Sales
0,2015,479574.8361
1,2016,459436.0054
2,2017,600192.5500


In [35]:
# add segment column


In [36]:
colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]

# Create the bar chart
fig = px.bar(df_yearly, x='year', y='Sales', color_discrete_sequence=colors)

# Set the title and axis labels
fig.update_layout(title='Sales per year', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=800, height=500)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()

In [37]:
colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]
#color_scale = px.colors.sequential.Viridis

df_yearly_segment = df_train.groupby(['year','Segment'])['Sales'].sum().reset_index()
# Create the bar chart
fig = px.bar(df_yearly_segment, x='year', y='Sales', color='Segment', color_discrete_sequence=colors)

# Set the title and axis labels
fig.update_layout(title='Sales per year by segment', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=800, height=500)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()


In [38]:
df_yearly_segment.head()

,year,Segment,Sales
0,2015,Consumer,262956.8006
1,2015,Corporate,127797.4957
2,2015,Home Office,88820.5398
3,2016,Consumer,265356.2933
4,2016,Corporate,119675.5989


In [39]:
colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]
#color_scale = px.colors.sequential.Viridis

df_yearly_region = df_train.groupby(['year','Region'])['Sales'].sum().reset_index()
# Create the bar chart
fig = px.bar(df_yearly_region, x='year', y='Sales', color='Region', color_discrete_sequence=colors)

# Set the title and axis labels
fig.update_layout(title='Sales per year by region', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=800, height=500)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()

In [40]:
colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]
#color_scale = px.colors.sequential.Viridis

df_yearly_region = df_train.groupby(['year','Category'])['Sales'].sum().reset_index()
# Create the bar chart
fig = px.bar(df_yearly_region, x='year', y='Sales', color='Category', color_discrete_sequence=colors)

# Set the title and axis labels
fig.update_layout(title='Sales per year by Category', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=800, height=500)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()

In [41]:
#colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]
color_scale = px.colors.sequential.Viridis

# Group the data by year and sub-category and compute the sum of sales for each group
df_yearly_subcat = df_train.groupby(['year', 'Sub-Category'])['Sales'].sum().reset_index()

# Get the top 3 best-selling sub-categories for each year
top3_subcats_by_year = df_yearly_subcat.groupby('year').apply(lambda x: x.nlargest(3, 'Sales')).reset_index(drop=True)

# Sort the data by year and sales in descending order
top3_subcats_by_year = top3_subcats_by_year.sort_values(['year', 'Sales'], ascending=[True, False])

# Create the bar chart
fig = px.bar(top3_subcats_by_year, x='year', y='Sales', color='Sub-Category', color_discrete_sequence=color_scale)

# Set the title and axis labels
fig.update_layout(title='Top 3 best-selling sub-categories per year', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=800, height=500)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()


In [42]:
#colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]

# Group the data by year and city and compute the sum of sales for each group
df_yearly_city = df_train.groupby(['year', 'City'])['Sales'].sum().reset_index()

# Get the top 3 best-selling cities for each year
top3_cities_by_year = df_yearly_city.groupby('year').apply(lambda x: x.nlargest(3, 'Sales')).reset_index(drop=True)

# Sort the data by year and sales in descending order
top3_cities_by_year = top3_cities_by_year.sort_values(['year', 'Sales'], ascending=[True, False])

# Create the bar chart
fig = px.bar(top3_cities_by_year, x='year', y='Sales', color='City', color_discrete_sequence=color_scale)

# Set the title and axis labels
fig.update_layout(title='Top 3 best-selling cities per year', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=800, height=500)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()


In [43]:
colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]

# Group the data by year and state and compute the sum of sales for each group
df_yearly_state = df_train.groupby(['year', 'State'])['Sales'].sum().reset_index()

# Get the top 3 best-selling states for each year
top3_states_by_year = df_yearly_state.groupby('year').apply(lambda x: x.nlargest(3, 'Sales')).reset_index(drop=True)

# Sort the data by year and sales in descending order
top3_states_by_year = top3_states_by_year.sort_values(['year', 'Sales'], ascending=[True, False])

# Create the bar chart
fig = px.bar(top3_states_by_year, x='year', y='Sales', color='State', color_discrete_sequence=colors)

# Set the title and axis labels
fig.update_layout(title='Top 3 best-selling states per year', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=800, height=500)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()


In [44]:
#colors = ["#00a0b0", "#6a4a3c", "#cc333f", "#eb6841"]

# Group the data by year and product name and compute the sum of sales for each group
df_yearly_product = df_train.groupby(['year', 'Product Name'])['Sales'].sum().reset_index()

# Get the top 3 best-selling product names for each year
top3_products_by_year = df_yearly_product.groupby('year').apply(lambda x: x.nlargest(3, 'Sales')).reset_index(drop=True)

# Sort the data by year and sales in descending order
top3_products_by_year = top3_products_by_year.sort_values(['year', 'Sales'], ascending=[True, False])

# Create the bar chart
fig = px.bar(top3_products_by_year, x='year', y='Sales', color='Product Name', color_discrete_sequence=color_scale)

# Set the title and axis labels
fig.update_layout(title='Top 3 best-selling products per year', xaxis_title='Year', yaxis_title='Sales')

# Set the size of the chart
fig.update_layout(width=1200, height=800)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')

# Show the chart
fig.show()


In [45]:
df_train['Order Period'] = df_train['Order Date'].dt.to_period('M') # convert to period

df_monthly_sales = df_train.groupby('Order Period')['Sales'].sum().reset_index()



/var/folders/0b/3m1h5_mj5q57nkn5qzvf49cr0000gn/T/ipykernel_2275/3701422579.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
df_monthly_sales['Month'] = df_monthly_sales['Order Period'].astype(str)

In [47]:
fig = px.bar(df_monthly_sales, x='Month', y='Sales', title='Total Monthly Sales',color='Sales', color_continuous_scale=px.colors.sequential.Blues)
fig.update_xaxes(type='category') # set x-axis type to category
fig.update_layout(width=1200, height=800)

# Add horizontal grid lines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')


fig.show()

In [48]:
df['Day of Week'] = df['Order Date'].dt.strftime('%A')

In [49]:
# plot best day of the week to buy  and display in descending order
fig = px.bar(df.groupby('Day of Week')['Sales'].sum().sort_values(ascending=False).reset_index(), x='Day of Week', y='Sales', title='Total Sales by Day of Week',color='Sales', color_continuous_scale=px.colors.sequential.Blues)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Set the background color
fig.update_layout(plot_bgcolor='white')
fig.update_layout(width=1200, height=800)
fig.show()

